In [3]:

import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [4]:
dataset = 'MV2014'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/e22-sup-tables1.csv', header=0)
print('Raw data loaded.')
data

Loading raw data for MV2014 ...
Raw data loaded.


,id,Proteins,Protein names,Gene names,Amino acid,Positions within proteins,Sequence window,Modified sequence,Known site (PhosphoSite Plus)?,Known site (Uniprot)?,...,Ratio H/L normalized B1___3 (30 min),Ratio L/M normalized B2___1 (30 min),Ratio L/M normalized B2___2 (30 min),Ratio L/M normalized B2___3 (30 min),Ratio M/H normalized B3___1 (30 min),Ratio M/H normalized B3___2 (30 min),Ratio M/H normalized B3___3 (30 min),PEP (Posterior error probability),Andromeda Score,Andromeda Score for localization
0,0,A0FGR8-6;A0FGR8;A0FGR8-2;A0FGR8-5;F2Z3K9;A6NFV7,Extended synaptotagmin-2,ESYT2;ESYT2;ESYT2,S,776;755;727;162;162;162,LLASPGHISVKEPTPSIASDISLPIATQELR,_EPTPS(ph)IAS(ph)DIS(ph)LPIATQELR_,yes,yes,...,NaN,NaN,0.076173,-0.303927,NaN,0.428468,NaN,1.350000e-44,131.620,131.620
1,1,A0FGR8-6;A0FGR8;A0FGR8-2;A0FGR8-5;F2Z3K9;A6NFV7,Extended synaptotagmin-2,ESYT2;ESYT2;ESYT2,S,779;758;730;165;165;165,SPGHISVKEPTPSIASDISLPIATQELRQRL,_EPTPS(ph)IAS(ph)DIS(ph)LPIATQELR_,yes,yes,...,NaN,NaN,0.069240,-0.303927,NaN,0.270017,NaN,2.680000e-132,197.040,131.620
2,2,A0FGR8-6;A0FGR8;A0FGR8-2;A0FGR8-5;F2Z3K9;A6NFV7,Extended synaptotagmin-2,ESYT2;ESYT2;ESYT2,S,782;761;733;168;168;168,HISVKEPTPSIASDISLPIATQELRQRLRQL,_EPTPSIAS(ph)DIS(ph)LPIATQELR_,yes,yes,...,NaN,NaN,-0.068602,-0.303927,NaN,0.404535,NaN,2.680000e-132,197.040,178.770
3,3,A0FGR8-6;A0FGR8;A0FGR8-2;A0FGR8-5;F2Z3K9;A6NFV7,Extended synaptotagmin-2,ESYT2;ESYT2;ESYT2,S,712;691;663;98;98;98,SVSKEGRKTSIKSHMSGSPGPGGSNTAPSTP,_SHMS(ph)GS(ph)PGPGGSNTAPSTPVIGGSDKPGMEEK_,yes,yes,...,NaN,NaN,0.414846,NaN,NaN,0.059224,NaN,1.800000e-103,149.840,132.070
4,4,A0FGR8-6;A0FGR8;A0FGR8-2;A0FGR8-5;F2Z3K9;A6NFV7,Extended synaptotagmin-2,ESYT2;ESYT2;ESYT2,S,714;693;665;100;100;100,SKEGRKTSIKSHMSGSPGPGGSNTAPSTPVI,_SHMS(ph)GS(ph)PGPGGSNTAPSTPVIGGSDKPGMEEK_,yes,NaN,...,NaN,NaN,0.414846,NaN,NaN,0.059224,NaN,1.800000e-103,149.840,149.840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10338,13193,Q93052;C9JUT4;C9JXK9,Lipoma-preferred partner,LPP;LPP;LPP,Y,317;317;154,AAGPGYGGRNDSDPTYGQQGHPNTWKREPGY,_NDSDPTY(ph)GQQGHPNTWK_,yes,yes,...,NaN,NaN,NaN,NaN,0.612020,NaN,NaN,1.790000e-41,128.590,79.317
10339,13195,Q9BWU0,Kanadaptin,SLC4A1AP,Y,464,VSRKRKAKNWEDEDFYDSDDDTFLDRTGLIE,_NWEDEDFY(ph)DSDDDTFLDR_,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.430000e-10,83.758,83.758
10340,13196,Q9NPQ8-3;Q9NPQ8;Q9NPQ8-4;Q9NPQ8-2;H0YE35;H0YC88,Synembryn-A,RIC8A;RIC8A;RIC8A,Y,441;435;434;429;43;21,LAARGLMAGGRPEGQYSEDEDTDTDEYKEAK,_GLM(ox)AGGRPEGQY(ph)SEDEDT(ph)DTDEYKEAK_,yes,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.390000e-29,93.505,93.505
10341,13198,Q9NRY4;Q9NRY4-2,Rho GTPase-activating protein 35,ARHGAP35,Y,1105;1105,PMDAVVKPRNEEENIYSVPHDSTQGKIITIR,_NEEENIY(ph)SVPHDSTQGK_,yes,yes,...,NaN,NaN,NaN,NaN,0.041759,NaN,NaN,1.990000e-22,106.200,106.200


In [5]:
data.columns = data.columns.str.strip()

In [6]:
# filter data to keep only those with localization probability >= 0.85
data = data[data['Localization prob'] >= 0.85] 

# Filtering out semi-colons from 'Amino acid', 'Positions within proteins', and 'Gene names' columns
data = data[~data['Amino acid'].str.contains(';', na=False)]
data = data[~data['Positions within proteins'].str.contains(';', na=False)]
data = data[~data['Gene names'].str.contains(';', na=False)]
data

,id,Proteins,Protein names,Gene names,Amino acid,Positions within proteins,Sequence window,Modified sequence,Known site (PhosphoSite Plus)?,Known site (Uniprot)?,...,Ratio H/L normalized B1___3 (30 min),Ratio L/M normalized B2___1 (30 min),Ratio L/M normalized B2___2 (30 min),Ratio L/M normalized B2___3 (30 min),Ratio M/H normalized B3___1 (30 min),Ratio M/H normalized B3___2 (30 min),Ratio M/H normalized B3___3 (30 min),PEP (Posterior error probability),Andromeda Score,Andromeda Score for localization
7,9,A0JLT2,Mediator of RNA polymerase II transcription su...,MED19,S,226,RKRKKKEKKKKKNRHSPDHPGMGSSQASSSS,_NRHS(ph)PDHPGMGSSQASSSSSLR_,yes,yes,...,NaN,0.007725,NaN,NaN,NaN,NaN,NaN,2.300000e-60,142.460,142.460
14,16,A0PJX4,Protein shisa-3 homolog,SHISA3,S,137,TCLRPKEPSQQPIRFSLRSYQTETLPMILTS,_EPSQQPIRFS(ph)LR_,NaN,NaN,...,NaN,NaN,NaN,NaN,0.553405,NaN,NaN,2.510000e-12,110.980,100.540
31,34,A1X283,SH3 and PX domain-containing protein 2B,SH3PXD2B,S,491,GVMDSGLPWSKDWKGSKDVLRKASSDMSASA,_DWKGS(ph)KDVLR_,NaN,NaN,...,NaN,-0.100036,NaN,NaN,-0.261832,NaN,NaN,1.670000e-06,120.840,90.707
32,35,A1X283,SH3 and PX domain-containing protein 2B,SH3PXD2B,S,528,DPDMEEKPSLPPRKESIIKSEGELLERERER,_KES(ph)IIKSEGELLER_,NaN,yes,...,NaN,-0.447632,NaN,NaN,0.640456,NaN,NaN,5.480000e-73,203.060,189.550
58,63,A2RU67,Uncharacterized protein KIAA1467,KIAA1467,S,62,VKNGKSPLGEAPEPDSDAEVAEAAKPHLSEV,_SPLGEAPEPDS(ph)DAEVAEAAKPHLSEVTTEGYPSEPLGGLEQK_,yes,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.710000e-69,120.710,120.710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10324,13169,P48681,Nestin,NES,Y,928,SQRNLEEEENLGKGEYQESLRSLEEEGQELP,_NLEEEENLGKGEY(ph)QESLRS(ph)LEEEGQELPQSADVQR_,yes,NaN,...,NaN,NaN,-0.029418,NaN,NaN,1.140794,NaN,4.970000e-48,107.710,107.710
10329,13177,Q03135,Caveolin-1,CAV1,Y,6,__________MSGGKYVDSEGHLYTVPIREQ,_Y(ph)VDSEGHLY(ph)TVPIR_,yes,yes,...,NaN,1.874402,NaN,NaN,NaN,1.930875,NaN,6.130000e-17,103.210,103.210
10330,13178,Q03135,Caveolin-1,CAV1,Y,14,__MSGGKYVDSEGHLYTVPIREQGNIYKPNN,_YVDSEGHLY(ph)TVPIR_,yes,yes,...,NaN,1.448194,NaN,NaN,1.684457,1.930875,NaN,3.250000e-27,121.760,105.140
10332,13184,Q13595,Transformer-2 protein homolog alpha,TRA2A,Y,87,RSHSHSHRRRSRSRSYTPEYRRRRSRSHSPM,_S(ph)RS(ph)RS(ph)Y(ph)TPEYR_,yes,NaN,...,-0.086844,NaN,NaN,NaN,NaN,NaN,NaN,2.670000e-05,90.453,87.429


In [7]:
# filter data
data['Sequence window'] = data['Sequence window'].str.replace('_', '')
data

,id,Proteins,Protein names,Gene names,Amino acid,Positions within proteins,Sequence window,Modified sequence,Known site (PhosphoSite Plus)?,Known site (Uniprot)?,...,Ratio H/L normalized B1___3 (30 min),Ratio L/M normalized B2___1 (30 min),Ratio L/M normalized B2___2 (30 min),Ratio L/M normalized B2___3 (30 min),Ratio M/H normalized B3___1 (30 min),Ratio M/H normalized B3___2 (30 min),Ratio M/H normalized B3___3 (30 min),PEP (Posterior error probability),Andromeda Score,Andromeda Score for localization
7,9,A0JLT2,Mediator of RNA polymerase II transcription su...,MED19,S,226,RKRKKKEKKKKKNRHSPDHPGMGSSQASSSS,_NRHS(ph)PDHPGMGSSQASSSSSLR_,yes,yes,...,NaN,0.007725,NaN,NaN,NaN,NaN,NaN,2.300000e-60,142.460,142.460
14,16,A0PJX4,Protein shisa-3 homolog,SHISA3,S,137,TCLRPKEPSQQPIRFSLRSYQTETLPMILTS,_EPSQQPIRFS(ph)LR_,NaN,NaN,...,NaN,NaN,NaN,NaN,0.553405,NaN,NaN,2.510000e-12,110.980,100.540
31,34,A1X283,SH3 and PX domain-containing protein 2B,SH3PXD2B,S,491,GVMDSGLPWSKDWKGSKDVLRKASSDMSASA,_DWKGS(ph)KDVLR_,NaN,NaN,...,NaN,-0.100036,NaN,NaN,-0.261832,NaN,NaN,1.670000e-06,120.840,90.707
32,35,A1X283,SH3 and PX domain-containing protein 2B,SH3PXD2B,S,528,DPDMEEKPSLPPRKESIIKSEGELLERERER,_KES(ph)IIKSEGELLER_,NaN,yes,...,NaN,-0.447632,NaN,NaN,0.640456,NaN,NaN,5.480000e-73,203.060,189.550
58,63,A2RU67,Uncharacterized protein KIAA1467,KIAA1467,S,62,VKNGKSPLGEAPEPDSDAEVAEAAKPHLSEV,_SPLGEAPEPDS(ph)DAEVAEAAKPHLSEVTTEGYPSEPLGGLEQK_,yes,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.710000e-69,120.710,120.710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10324,13169,P48681,Nestin,NES,Y,928,SQRNLEEEENLGKGEYQESLRSLEEEGQELP,_NLEEEENLGKGEY(ph)QESLRS(ph)LEEEGQELPQSADVQR_,yes,NaN,...,NaN,NaN,-0.029418,NaN,NaN,1.140794,NaN,4.970000e-48,107.710,107.710
10329,13177,Q03135,Caveolin-1,CAV1,Y,6,MSGGKYVDSEGHLYTVPIREQ,_Y(ph)VDSEGHLY(ph)TVPIR_,yes,yes,...,NaN,1.874402,NaN,NaN,NaN,1.930875,NaN,6.130000e-17,103.210,103.210
10330,13178,Q03135,Caveolin-1,CAV1,Y,14,MSGGKYVDSEGHLYTVPIREQGNIYKPNN,_YVDSEGHLY(ph)TVPIR_,yes,yes,...,NaN,1.448194,NaN,NaN,1.684457,1.930875,NaN,3.250000e-27,121.760,105.140
10332,13184,Q13595,Transformer-2 protein homolog alpha,TRA2A,Y,87,RSHSHSHRRRSRSRSYTPEYRRRRSRSHSPM,_S(ph)RS(ph)RS(ph)Y(ph)TPEYR_,yes,NaN,...,-0.086844,NaN,NaN,NaN,NaN,NaN,NaN,2.670000e-05,90.453,87.429


In [8]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    

    fasta_sequence = list(SeqIO.parse(open(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta'), "fasta"))
    
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                # gene_name_match = re.search("GN=(\w+)", seq_record.description)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else: 
                    print(f"No gene name found in description for sequence: {i_str}")
    
    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')
    return dataset 
    


In [9]:
data = match_seq_to_genename(data, 'Sequence window')

RKRKKKEKKKKKNRHSPDHPGMGSSQASSSS
Match found for sequence: ID: sp|A0JLT2|MED19_HUMAN
Name: sp|A0JLT2|MED19_HUMAN
Description: sp|A0JLT2|MED19_HUMAN Mediator of RNA polymerase II transcription subunit 19 OS=Homo sapiens OX=9606 GN=MED19 PE=1 SV=2
Number of features: 0
Seq('MENFTALFGAQADPPPPPTALGFGPGKPPPPPPPPAGGGPGTAPPPTAATAPPG...SLR')
Gene name match: <re.Match object; span=(101, 109), match='GN=MED19'>
Match found: RKRKKKEKKKKKNRHSPDHPGMGSSQASSSS -> MED19
TCLRPKEPSQQPIRFSLRSYQTETLPMILTS
Match found for sequence: ID: sp|A0PJX4|SHSA3_HUMAN
Name: sp|A0PJX4|SHSA3_HUMAN
Description: sp|A0PJX4|SHSA3_HUMAN Protein shisa-3 homolog OS=Homo sapiens OX=9606 GN=SHISA3 PE=1 SV=1
Number of features: 0
Seq('MRALLALCLLLGWLRWGPAGAQQSGEYCHGWVDVQGNYHEGFQCPEDFDTLDAT...SSS')
Gene name match: <re.Match object; span=(70, 79), match='GN=SHISA3'>
Match found: TCLRPKEPSQQPIRFSLRSYQTETLPMILTS -> SHISA3
GVMDSGLPWSKDWKGSKDVLRKASSDMSASA
Match found for sequence: ID: sp|A1X283|SPD2B_HUMAN
Name: sp|A1X283|SPD2B_HUMAN


In [10]:
data

,id,Proteins,Protein names,Gene names,Amino acid,Positions within proteins,Sequence window,Modified sequence,Known site (PhosphoSite Plus)?,Known site (Uniprot)?,...,Ratio L/M normalized B2___1 (30 min),Ratio L/M normalized B2___2 (30 min),Ratio L/M normalized B2___3 (30 min),Ratio M/H normalized B3___1 (30 min),Ratio M/H normalized B3___2 (30 min),Ratio M/H normalized B3___3 (30 min),PEP (Posterior error probability),Andromeda Score,Andromeda Score for localization,GeneName
7,9,A0JLT2,Mediator of RNA polymerase II transcription su...,MED19,S,226,RKRKKKEKKKKKNRHSPDHPGMGSSQASSSS,_NRHS(ph)PDHPGMGSSQASSSSSLR_,yes,yes,...,0.007725,NaN,NaN,NaN,NaN,NaN,2.300000e-60,142.460,142.460,MED19
14,16,A0PJX4,Protein shisa-3 homolog,SHISA3,S,137,TCLRPKEPSQQPIRFSLRSYQTETLPMILTS,_EPSQQPIRFS(ph)LR_,NaN,NaN,...,NaN,NaN,NaN,0.553405,NaN,NaN,2.510000e-12,110.980,100.540,SHISA3
31,34,A1X283,SH3 and PX domain-containing protein 2B,SH3PXD2B,S,491,GVMDSGLPWSKDWKGSKDVLRKASSDMSASA,_DWKGS(ph)KDVLR_,NaN,NaN,...,-0.100036,NaN,NaN,-0.261832,NaN,NaN,1.670000e-06,120.840,90.707,SH3PXD2B
32,35,A1X283,SH3 and PX domain-containing protein 2B,SH3PXD2B,S,528,DPDMEEKPSLPPRKESIIKSEGELLERERER,_KES(ph)IIKSEGELLER_,NaN,yes,...,-0.447632,NaN,NaN,0.640456,NaN,NaN,5.480000e-73,203.060,189.550,SH3PXD2B
58,63,A2RU67,Uncharacterized protein KIAA1467,KIAA1467,S,62,VKNGKSPLGEAPEPDSDAEVAEAAKPHLSEV,_SPLGEAPEPDS(ph)DAEVAEAAKPHLSEVTTEGYPSEPLGGLEQK_,yes,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,2.710000e-69,120.710,120.710,FAM234B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10324,13169,P48681,Nestin,NES,Y,928,SQRNLEEEENLGKGEYQESLRSLEEEGQELP,_NLEEEENLGKGEY(ph)QESLRS(ph)LEEEGQELPQSADVQR_,yes,NaN,...,NaN,-0.029418,NaN,NaN,1.140794,NaN,4.970000e-48,107.710,107.710,NES
10329,13177,Q03135,Caveolin-1,CAV1,Y,6,MSGGKYVDSEGHLYTVPIREQ,_Y(ph)VDSEGHLY(ph)TVPIR_,yes,yes,...,1.874402,NaN,NaN,NaN,1.930875,NaN,6.130000e-17,103.210,103.210,CAV1
10330,13178,Q03135,Caveolin-1,CAV1,Y,14,MSGGKYVDSEGHLYTVPIREQGNIYKPNN,_YVDSEGHLY(ph)TVPIR_,yes,yes,...,1.448194,NaN,NaN,1.684457,1.930875,NaN,3.250000e-27,121.760,105.140,CAV1
10332,13184,Q13595,Transformer-2 protein homolog alpha,TRA2A,Y,87,RSHSHSHRRRSRSRSYTPEYRRRRSRSHSPM,_S(ph)RS(ph)RS(ph)Y(ph)TPEYR_,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.670000e-05,90.453,87.429,TRA2A


In [11]:
# Ensure GeneName exists before proceeding
if 'GeneName' not in data.columns:
    raise ValueError("GeneName column is missing! Check match_seq_to_genename function.")


In [12]:
data['Phosphosite'] = data['Amino acid'].astype(str) + '(' + data['Positions within proteins'].astype(str) + ')'

# Debugging: Check if 'Phosphosite' column is present after creation
if 'Phosphosite' not in data.columns:
    print("Error: 'Phosphosite' column not created!")
else:
    print("Phosphosite column created successfully.")

data

Phosphosite column created successfully.


,id,Proteins,Protein names,Gene names,Amino acid,Positions within proteins,Sequence window,Modified sequence,Known site (PhosphoSite Plus)?,Known site (Uniprot)?,...,Ratio L/M normalized B2___2 (30 min),Ratio L/M normalized B2___3 (30 min),Ratio M/H normalized B3___1 (30 min),Ratio M/H normalized B3___2 (30 min),Ratio M/H normalized B3___3 (30 min),PEP (Posterior error probability),Andromeda Score,Andromeda Score for localization,GeneName,Phosphosite
7,9,A0JLT2,Mediator of RNA polymerase II transcription su...,MED19,S,226,RKRKKKEKKKKKNRHSPDHPGMGSSQASSSS,_NRHS(ph)PDHPGMGSSQASSSSSLR_,yes,yes,...,NaN,NaN,NaN,NaN,NaN,2.300000e-60,142.460,142.460,MED19,S(226)
14,16,A0PJX4,Protein shisa-3 homolog,SHISA3,S,137,TCLRPKEPSQQPIRFSLRSYQTETLPMILTS,_EPSQQPIRFS(ph)LR_,NaN,NaN,...,NaN,NaN,0.553405,NaN,NaN,2.510000e-12,110.980,100.540,SHISA3,S(137)
31,34,A1X283,SH3 and PX domain-containing protein 2B,SH3PXD2B,S,491,GVMDSGLPWSKDWKGSKDVLRKASSDMSASA,_DWKGS(ph)KDVLR_,NaN,NaN,...,NaN,NaN,-0.261832,NaN,NaN,1.670000e-06,120.840,90.707,SH3PXD2B,S(491)
32,35,A1X283,SH3 and PX domain-containing protein 2B,SH3PXD2B,S,528,DPDMEEKPSLPPRKESIIKSEGELLERERER,_KES(ph)IIKSEGELLER_,NaN,yes,...,NaN,NaN,0.640456,NaN,NaN,5.480000e-73,203.060,189.550,SH3PXD2B,S(528)
58,63,A2RU67,Uncharacterized protein KIAA1467,KIAA1467,S,62,VKNGKSPLGEAPEPDSDAEVAEAAKPHLSEV,_SPLGEAPEPDS(ph)DAEVAEAAKPHLSEVTTEGYPSEPLGGLEQK_,yes,yes,...,NaN,NaN,NaN,NaN,NaN,2.710000e-69,120.710,120.710,FAM234B,S(62)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10324,13169,P48681,Nestin,NES,Y,928,SQRNLEEEENLGKGEYQESLRSLEEEGQELP,_NLEEEENLGKGEY(ph)QESLRS(ph)LEEEGQELPQSADVQR_,yes,NaN,...,-0.029418,NaN,NaN,1.140794,NaN,4.970000e-48,107.710,107.710,NES,Y(928)
10329,13177,Q03135,Caveolin-1,CAV1,Y,6,MSGGKYVDSEGHLYTVPIREQ,_Y(ph)VDSEGHLY(ph)TVPIR_,yes,yes,...,NaN,NaN,NaN,1.930875,NaN,6.130000e-17,103.210,103.210,CAV1,Y(6)
10330,13178,Q03135,Caveolin-1,CAV1,Y,14,MSGGKYVDSEGHLYTVPIREQGNIYKPNN,_YVDSEGHLY(ph)TVPIR_,yes,yes,...,NaN,NaN,1.684457,1.930875,NaN,3.250000e-27,121.760,105.140,CAV1,Y(14)
10332,13184,Q13595,Transformer-2 protein homolog alpha,TRA2A,Y,87,RSHSHSHRRRSRSRSYTPEYRRRRSRSHSPM,_S(ph)RS(ph)RS(ph)Y(ph)TPEYR_,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,2.670000e-05,90.453,87.429,TRA2A,Y(87)


In [13]:
# Keep only 'Phosphosite' and ratio columns
keepcols = ['Phosphosite'] + ['GeneName'] + [col for col in data.columns if 'Ratio' in col]
data = data[keepcols]
data

,Phosphosite,GeneName,Ratio M/L normalized A1___1 (2 min),Ratio M/L normalized A1___2 (2 min),Ratio M/L normalized A1___3 (2 min),Ratio H/M normalized A2___1 (2 min),Ratio H/M normalized A2___2 (2 min),Ratio H/M normalized A2___3 (2 min),Ratio L/H normalized A3___1 (2 min),Ratio L/H normalized A3___2 (2 min),...,Ratio L/H normalized B3___3 (5 min),Ratio H/L normalized B1___1 (30 min),Ratio H/L normalized B1___2 (30 min),Ratio H/L normalized B1___3 (30 min),Ratio L/M normalized B2___1 (30 min),Ratio L/M normalized B2___2 (30 min),Ratio L/M normalized B2___3 (30 min),Ratio M/H normalized B3___1 (30 min),Ratio M/H normalized B3___2 (30 min),Ratio M/H normalized B3___3 (30 min)
7,S(226),MED19,0.176067,NaN,NaN,0.040963,NaN,NaN,NaN,NaN,...,NaN,-0.117099,NaN,NaN,0.007725,NaN,NaN,NaN,NaN,NaN
14,S(137),SHISA3,NaN,NaN,NaN,1.026234,NaN,NaN,0.838074,NaN,...,NaN,0.712112,NaN,NaN,NaN,NaN,NaN,0.553405,NaN,NaN
31,S(491),SH3PXD2B,-1.560728,NaN,NaN,-1.732932,NaN,NaN,NaN,NaN,...,NaN,-0.074745,NaN,NaN,-0.100036,NaN,NaN,-0.261832,NaN,NaN
32,S(528),SH3PXD2B,1.245496,NaN,NaN,1.048934,NaN,NaN,NaN,NaN,...,NaN,0.971663,NaN,NaN,-0.447632,NaN,NaN,0.640456,NaN,NaN
58,S(62),FAM234B,-0.234720,NaN,NaN,0.042644,NaN,NaN,-0.512884,NaN,...,NaN,0.015498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10324,Y(928),NES,NaN,NaN,NaN,NaN,0.412402,NaN,NaN,0.212835,...,NaN,NaN,1.483519,NaN,NaN,-0.029418,NaN,NaN,1.140794,NaN
10329,Y(6),CAV1,NaN,NaN,NaN,-0.120420,NaN,NaN,0.367601,NaN,...,NaN,2.120750,1.665802,NaN,1.874402,NaN,NaN,NaN,1.930875,NaN
10330,Y(14),CAV1,NaN,NaN,NaN,0.689031,NaN,NaN,0.323371,NaN,...,NaN,1.933006,1.665802,NaN,1.448194,NaN,NaN,1.684457,1.930875,NaN
10332,Y(87),TRA2A,NaN,NaN,2.134878,NaN,NaN,-1.711842,NaN,NaN,...,NaN,NaN,NaN,-0.086844,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
print("Data after subsetting columns:", data)


Data after subsetting columns:       Phosphosite  GeneName  Ratio M/L normalized A1___1 (2 min)  \
7          S(226)     MED19                             0.176067   
14         S(137)    SHISA3                                  NaN   
31         S(491)  SH3PXD2B                            -1.560728   
32         S(528)  SH3PXD2B                             1.245496   
58          S(62)   FAM234B                            -0.234720   
...           ...       ...                                  ...   
10324      Y(928)       NES                                  NaN   
10329        Y(6)      CAV1                                  NaN   
10330       Y(14)      CAV1                                  NaN   
10332       Y(87)     TRA2A                                  NaN   
10339      Y(464)  SLC4A1AP                            -0.330157   

       Ratio M/L normalized A1___2 (2 min)  \
7                                      NaN   
14                                     NaN   
31            

In [15]:
print("Cols after subsetting:", data.columns)

Cols after subsetting: Index(['Phosphosite', 'GeneName', 'Ratio M/L normalized A1___1 (2 min)',
       'Ratio M/L normalized A1___2 (2 min)',
       'Ratio M/L normalized A1___3 (2 min)',
       'Ratio H/M normalized A2___1 (2 min)',
       'Ratio H/M normalized A2___2 (2 min)',
       'Ratio H/M normalized A2___3 (2 min)',
       'Ratio L/H normalized A3___1 (2 min)',
       'Ratio L/H normalized A3___2 (2 min)',
       'Ratio L/H normalized A3___3 (2 min)',
       'Ratio H/L normalized A1___1 (10 min)',
       'Ratio H/L normalized A1___2 (10 min)',
       'Ratio H/L normalized A1___3 (10 min)',
       'Ratio L/M normalized A2___1 (10 min)',
       'Ratio L/M normalized A2___2 (10 min)',
       'Ratio L/M normalized A2___3 (10 min)',
       'Ratio M/H normalized A3___1 (10 min)',
       'Ratio M/H normalized A3___2 (10 min)',
       'Ratio M/H normalized A3___3 (10 min)',
       'Ratio M/L normalized B1___1 (5 min)',
       'Ratio M/L normalized B1___2 (5 min)',
       'Ratio M/L nor

In [16]:
# log2 transform the ratio columns (Ratio columns)
Ratio_columns = [col for col in data.columns if 'Ratio' in col]
data[Ratio_columns] = data[Ratio_columns].apply(pd.to_numeric, errors='coerce')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_1319/1446063117.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[Ratio_columns] = data[Ratio_columns].apply(pd.to_numeric, errors='coerce')


In [17]:
def log2_transform(dataset):
    '''
    Log2 transform a dataset.
    
    args:
    =====
    dataset: <pd.Dataframe>
    
    out:
    ====
    dataset: <pd.Dataframe> with log2 transformed values

    '''
    cols_to_transform = dataset[Ratio_columns]
    dataset[Ratio_columns] = cols_to_transform.apply(np.log2)
    print('Data has been log2 transformed.')
    return dataset


In [18]:
data = log2_transform(data)
print(f"DataFrame after log2 transformation:\n{data}") # Print the DataFrame after log2 transformation
data

Data has been log2 transformed.
DataFrame after log2 transformation:
      Phosphosite  GeneName  Ratio M/L normalized A1___1 (2 min)  \
7          S(226)     MED19                             -2.50580   
14         S(137)    SHISA3                                  NaN   
31         S(491)  SH3PXD2B                                  NaN   
32         S(528)  SH3PXD2B                              0.31672   
58          S(62)   FAM234B                                  NaN   
...           ...       ...                                  ...   
10324      Y(928)       NES                                  NaN   
10329        Y(6)      CAV1                                  NaN   
10330       Y(14)      CAV1                                  NaN   
10332       Y(87)     TRA2A                                  NaN   
10339      Y(464)  SLC4A1AP                                  NaN   

       Ratio M/L normalized A1___2 (2 min)  \
7                                      NaN   
14                    

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_1319/1555182160.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[Ratio_columns] = cols_to_transform.apply(np.log2)


,Phosphosite,GeneName,Ratio M/L normalized A1___1 (2 min),Ratio M/L normalized A1___2 (2 min),Ratio M/L normalized A1___3 (2 min),Ratio H/M normalized A2___1 (2 min),Ratio H/M normalized A2___2 (2 min),Ratio H/M normalized A2___3 (2 min),Ratio L/H normalized A3___1 (2 min),Ratio L/H normalized A3___2 (2 min),...,Ratio L/H normalized B3___3 (5 min),Ratio H/L normalized B1___1 (30 min),Ratio H/L normalized B1___2 (30 min),Ratio H/L normalized B1___3 (30 min),Ratio L/M normalized B2___1 (30 min),Ratio L/M normalized B2___2 (30 min),Ratio L/M normalized B2___3 (30 min),Ratio M/H normalized B3___1 (30 min),Ratio M/H normalized B3___2 (30 min),Ratio M/H normalized B3___3 (30 min)
7,S(226),MED19,-2.50580,NaN,NaN,-4.609550,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-7.016243,NaN,NaN,NaN,NaN,NaN
14,S(137),SHISA3,NaN,NaN,NaN,0.037360,NaN,NaN,-0.254850,NaN,...,NaN,-0.489825,NaN,NaN,NaN,NaN,NaN,-0.853593,NaN,NaN
31,S(491),SH3PXD2B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,S(528),SH3PXD2B,0.31672,NaN,NaN,0.068924,NaN,NaN,NaN,NaN,...,NaN,-0.041472,NaN,NaN,NaN,NaN,NaN,-0.642828,NaN,NaN
58,S(62),FAM234B,NaN,NaN,NaN,-4.551503,NaN,NaN,NaN,NaN,...,NaN,-6.011813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10324,Y(928),NES,NaN,NaN,NaN,NaN,-1.277876,NaN,NaN,-2.232196,...,NaN,NaN,0.569023,NaN,NaN,NaN,NaN,NaN,0.190038,NaN
10329,Y(6),CAV1,NaN,NaN,NaN,NaN,NaN,NaN,-1.443786,NaN,...,NaN,1.084575,0.736217,NaN,0.906430,NaN,NaN,NaN,0.949255,NaN
10330,Y(14),CAV1,NaN,NaN,NaN,-0.537360,NaN,NaN,-1.628737,NaN,...,NaN,0.950846,0.736217,NaN,0.534255,NaN,NaN,0.752284,0.949255,NaN
10332,Y(87),TRA2A,NaN,NaN,1.094154,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_1319/3553972964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


Phosphosite IDs created.
Phosphosite IDs created.


In [20]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case = False)]
    data = data[~data.phosphosite_ID.str.contains(';', case = False)] # remove rows containing ';' in phosphosite_ID column
    data = data[~data.phosphosite_ID.str.contains('-', case = False)] # remove rows containing '-' in phosphosite_ID column
    
    # check whether there are any phosphosites with multiple measurements
    data_grouped = data.groupby(by = 'phosphosite_ID')
    if len(data) != len(data_grouped):
        data = data_grouped.mean()
        data.reset_index(inplace=True) # reset index
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')
        
    print(data)
        
    data = data.replace([np.inf, -np.inf], np.nan)
        
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)
    return data

In [21]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

Phosphosites with multiple measurements have been averaged
     phosphosite_ID  Ratio M/L normalized A1___1 (2 min)  \
0       AAK1_S(846)                            -0.880185   
1       AATF_S(203)                                  NaN   
2       AATF_S(316)                                  NaN   
3       AATF_S(320)                                  NaN   
4       AATF_S(321)                                  NaN   
...             ...                                  ...   
2015  ZNF830_S(351)                                  NaN   
2016   ZRSR2_S(349)                            -4.253854   
2017  ZZEF1_S(1537)                                  NaN   
2018  ZZEF1_S(1540)                                  NaN   
2019  ZZEF1_S(2444)                            -2.897319   

      Ratio M/L normalized A1___2 (2 min)  \
0                                     NaN   
1                                     NaN   
2                                     NaN   
3                               -2.97291

,phosphosite_ID,Ratio M/L normalized A1___1 (2 min),Ratio M/L normalized A1___2 (2 min),Ratio M/L normalized A1___3 (2 min),Ratio H/M normalized A2___1 (2 min),Ratio H/M normalized A2___2 (2 min),Ratio H/M normalized A2___3 (2 min),Ratio L/H normalized A3___1 (2 min),Ratio L/H normalized A3___2 (2 min),Ratio L/H normalized A3___3 (2 min),...,Ratio L/H normalized B3___3 (5 min),Ratio H/L normalized B1___1 (30 min),Ratio H/L normalized B1___2 (30 min),Ratio H/L normalized B1___3 (30 min),Ratio L/M normalized B2___1 (30 min),Ratio L/M normalized B2___2 (30 min),Ratio L/M normalized B2___3 (30 min),Ratio M/H normalized B3___1 (30 min),Ratio M/H normalized B3___2 (30 min),Ratio M/H normalized B3___3 (30 min)
0,AAK1_S(846),-0.880185,NaN,NaN,NaN,NaN,NaN,-0.099847,NaN,NaN,...,NaN,1.377459,NaN,NaN,1.14895,NaN,NaN,1.686102,NaN,NaN
1,AATF_S(203),NaN,NaN,NaN,NaN,NaN,NaN,-4.295165,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AATF_S(316),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.914001,NaN,NaN,-5.202685,NaN,NaN,NaN,NaN,NaN,-3.319181
3,AATF_S(320),NaN,-2.972912,NaN,NaN,NaN,NaN,NaN,-3.77989,NaN,...,-5.914001,NaN,NaN,-5.202685,NaN,NaN,NaN,NaN,NaN,-3.319181
4,AATF_S(321),NaN,-2.972912,NaN,NaN,NaN,NaN,NaN,-3.77989,NaN,...,-5.914001,NaN,NaN,-5.202685,NaN,NaN,NaN,NaN,NaN,-3.319181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,ZNF830_S(351),NaN,NaN,NaN,NaN,NaN,NaN,-4.135504,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.021946,NaN,NaN
2016,ZRSR2_S(349),-4.253854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-5.328462,NaN,NaN,NaN,NaN,NaN,-4.484019,NaN,NaN
2017,ZZEF1_S(1537),NaN,0.623095,NaN,NaN,0.724298,NaN,NaN,NaN,NaN,...,NaN,NaN,1.287946,NaN,NaN,0.898213,NaN,NaN,1.433025,NaN
2018,ZZEF1_S(1540),NaN,0.623095,NaN,NaN,0.724298,NaN,NaN,NaN,NaN,...,NaN,NaN,1.287946,NaN,NaN,0.898213,NaN,NaN,1.433025,NaN


In [25]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/MV2014.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)

MV2014 has been saved to CSV successfully!      phosphosite_ID  Ratio M/L normalized A1___1 (2 min)  \
0       AAK1_S(846)                            -0.880185   
1       AATF_S(203)                                  NaN   
2       AATF_S(316)                                  NaN   
3       AATF_S(320)                                  NaN   
4       AATF_S(321)                                  NaN   
...             ...                                  ...   
2015  ZNF830_S(351)                                  NaN   
2016   ZRSR2_S(349)                            -4.253854   
2017  ZZEF1_S(1537)                                  NaN   
2018  ZZEF1_S(1540)                                  NaN   
2019  ZZEF1_S(2444)                            -2.897319   

      Ratio M/L normalized A1___2 (2 min)  \
0                                     NaN   
1                                     NaN   
2                                     NaN   
3                               -2.972912   
4          